## Methodology


 We want to extend the insights that Zeek and RITA are giving us about the probable sketchiness of particular connections. The goal in this notebook is to explain the data analysis in a way that other, more fluid, tools can be built. 
 

 
**RITAs statistical analysis plus locally gathered heuristics**   
 RITA and Zeek are fantastic tools to explore traffic and narrow down bad actors but published blacklists are always laggy and incomplete, when they work and are supported. Also - the scoring doesn't filter for items like DNS or other probable-good services (or call them out), which adds to the SecOps response time to filter.  Here we can bring some low-cost tools to bear and some local understanding to narrow down what to invest time into, good or bad.


Some methods we're using:
- Combine Beacons and Conns files to identify unique talkers
- retrieve BGP Autonomous Systems info to identify originators (or listeners) in Wild West areas

also - are we getting connections from nets that practice good hygiene? 
- does an ip have a DNS entry?
- does an ip have a PTR record?

What local tools can add dimension?
- was the sender identified as malicious by other means? (fail2ban ICMP type 3 returns)


Unfortunately these blacklists don't work anymore:
- MalwareDomains.com
- MalwareDomainList.com
- malware-domains.com

Fresher Blacklist providers (as of 8/2021):
- https://urlhaus.abuse.ch/downloads/csv/
- https://github.com/curbengh/urlhaus-filter
- https://github.com/StevenBlack/hosts
- https://www.abuseipdb.com


### running this with your own data:
- run the rita_extractor.sh to get your data
- put the records where the "load data" cell (should be \#4) can find them.
- you probably want to generate your own "beaconish_asns" file:
    - this is currently looking for a file in the records dir called 'beaconish_asns'. You can make your own with:
    ```
    df[(df['Score'] >.80)]['asn_desc']
    ```
    - scan the list to make sure that good services (like DNS, prometheus, etc) are srubbed
    - replace the contents of records/beaconish_asns with that output. If you put it anywhere else a function will break. 
- check the metadata sections for metadata lists. Heads up: the functions work with their own metadata lists. Changing the data in the cells won't change the way the functions operate. 
- If you want to publish you may want to use the obscureips script. 



**Home and Home Office Networks**  
I have a separate notebook for this that needs to be updated for the latest changes. I'll remove this when it's published.
You may want to analyze office or home net traffic and find out chatty corporate tools. You can add your own, but this will add a score to tag "friendly surveillance" from Apple, Google, et al. My lists are US-Centric - tailor to your locale. These may or may not be things you want in your custom RITA blacklist, but you may not know what they are yet. 



TODOs: 
- building this in a framework
- summarize outputs as a report (that could be used to kick off tickets, or add to a log)
- auto export the ranking to an API or datastore that other apps could use
- make the heuristics modular such that we can trivially import n-number of blacklists, etc
- add thread/async to dns/ptr lookups

In [1]:
# imports
import pandas as pd
import numpy as np

# Viz imports
import matplotlib.pyplot as plt
import seaborn as sns

# Config matplotlib
%matplotlib inline
plt.rcParams["patch.force_edgecolor"] = True # in matplotlib, edge borders are turned off by default.
sns.set_style("darkgrid") # set a grey grid as a background

# turn off warnings
import warnings
warnings.filterwarnings('ignore')

import csv
import json
import datetime as datetime
import time

# ip/AS lookup tools
import socket
from ipwhois import IPWhois
from ipwhois.net import Net
from ipwhois.asn import IPASN

In [2]:
# functions imports
from functions.functions import iplookup, getAsInfo, is_sketchy, is_corp, is_sketchy_provider, isip, has_dns, has_ptr, f2b_marked, in_blacklist, tally_total, total_score
    

### metadata

**functions calling these lists are replicated in the functions/functions.py file.**

The items are left here for clarity, but if you change them here the functions won't notice.

In [3]:
# define corporate target AS Descriptors
invasive_corps = ['AMAZON','APPLE','GOOGLE','MICROSOFT','CLOUDFLARENET','SALESFORCE','AKAMAI','OPENDNS']
# define countries with lax enforcement
sketchy_countries = ['CN','RU','VN','HK','TW','IN','BR','RO','HU','KR','IT','UG','TR','MY','BO','CO']
# define ISPs with lax enforcement
sketchy_providers = []
providers = open('records/beaconish_asns','r').readlines()
for p in providers:
    sketchy_providers.append(p.split()[0])
#
sketchy_providers[:5]

['AS-SONICTELECOM,', 'ASIANET', 'ASN-SPIN,', 'ASN-WINDTRE', 'BAIDU']

### load data

this takes the output of rita show-long-connections (dfconns) and rita show-beacons (dfbeacons).

The obscured IPs must be the same for each file - we merge the two on a matched ipsrc->ipdst key. The df will fail to create if there is nothing to merge on.

In [4]:
dfbeacons = pd.read_csv('records/scrubbed_ext_20210315062437_beacons.csv')
dfconns = pd.read_csv('records/scrubbed_ext_20210315062437_longconns.csv')
#dfdns = pd.read_csv('dns.csv')

### merge 

In [5]:
dfconns['ConnString'] = dfconns['Source IP'] + '->' + dfconns['Destination IP']
dfbeacons['ConnString'] = dfbeacons['Source IP'] + '->'+ dfbeacons['Destination IP']
df = pd.merge(dfbeacons, dfconns, on=['ConnString'], how='inner')

**clean up merge data**

since the merge key is an amagalm of connection to connection strings, the Source IP and Destination IP collision columns _x and _y should be erroneous.

In [6]:
del df['Source IP_y']
del df['Destination IP_y']
df.rename(columns={"Destination IP_x": "Destination IP",'Source IP_x':'Source IP'}, inplace=True)

In [7]:
df.columns

Index(['Score', 'Source IP', 'Destination IP', 'Connections', 'Avg Bytes',
       'Intvl Range', 'Size Range', 'Top Intvl', 'Top Size', 'Top Intvl Count',
       'Top Size Count', 'Intvl Skew', 'Size Skew', 'Intvl Dispersion',
       'Size Dispersion', 'ConnString', 'Port:Protocol:Service', 'Duration'],
      dtype='object')

### Describing the IP sets

Basic data analysis.

In [8]:
# unique localhosts
len(df['Source IP'].unique())

116

In [9]:
df['Source IP'].unique()

array(['65.254.18.118', '104.153.105.82', '192.168.23.89',
       '49.235.37.144', '42.192.234.75', '45.119.126.59', '120.70.98.132',
       '61.221.64.5', '202.95.14.159', '190.210.182.179',
       '106.124.131.214', '157.131.240.195', '49.232.39.21',
       '211.219.18.186', '123.206.90.149', '190.85.94.106',
       '49.233.77.12', '114.219.157.97', '210.178.94.227',
       '118.212.146.30', '106.75.61.147', '129.211.119.202',
       '35.220.253.166', '119.45.177.219', '125.5.180.84',
       '134.175.111.215', '134.209.41.198', '189.110.163.26',
       '116.228.233.91', '190.171.133.10', '144.91.70.164',
       '101.89.213.84', '118.98.96.184', '64.64.227.224',
       '222.168.18.227', '103.10.87.54', '203.195.207.85',
       '121.4.111.232', '101.36.127.150', '2.196.193.181',
       '123.127.244.100', '121.241.244.92', '182.254.149.130',
       '81.68.243.13', '106.124.142.64', '1.220.185.149', '123.30.149.34',
       '103.55.62.78', '118.24.121.227', '120.92.166.166',
       '121.4

In [10]:
# unique targets
len(df['Destination IP'].unique())

640

In [11]:
# unique connections
len(df['ConnString'].unique())

754

### Adding AS info

If you're reading this notebook I'm guessing that you probably already know what BGP is and get why we're grading  data in this way. If not, read on:

**Quick BGP/AS intro (stolen liberally from Cloudflare's great tutorial):**  
The *Border Gateway Protocol (BGP)* is the postal service of the Internet. To manage these endpoints each network broken into smaller networks known as *Autonomous systems (AS)*. Each of these networks is essentially a large pool of routers run by a single organization. 

If we continue to think of BGP as the postal service of the Internet, AS’s are like individual post office branches. A town may have hundreds of mailboxes, but the mail in those boxes must go through the local postal branch before being routed to another destination. The internal routers within an AS are like mailboxes, they forward their outbound transmissions to the AS, which then uses BGP routing to get these transmissions to their destinations.

To get on the Internet you need an IP Block, which needs a BGP AS. The companies that own the AS are responsible for the traffic that goes through them. If you have a lot of bad traffic coming from one server in an AS then there's reason to believe that blocks in other IP space controlled by that AS are also probably poorly managed. 

**Grading traffic from a particular AS block**  
This may be part of a decision to drop traffic coming from a single server or from the entire IP space as a Network Admin, but in this context we're simply going to grade traffic to that AS as more suspicious.

**Grading traffic coming from a Country**
Each BGP area is controlled within a Region, which then distributes to countries who have laws regarding internet traffic, hacking, etc. Some countries are more permissive than others with regard to hacking, fraud and spam. While it's incorrect and unfair to grade the citizens or services of a country based on the worst of their netizens, it's reasonable to grade countries with overly promiscuous (or non-existent) laws about hacking higher for further review. 

**Grading traffic coming from a Company**  
Much of the same rules apply here - if a company has a policy for their devices to send tracking data home through your networks you should be able to know about it. If they have lax policies concerning network access or services that could host C2 or bad traffic, you should be able to know about that, too. 

#### add ASN columns

takes a little time for the lookups

**loop through and add AS fields**

In [12]:
asdata = ['asn','asn_cidr','asn_country','asn_desc']
for a in asdata:
    df[a] =  df[['Source IP','Destination IP']].apply(getAsInfo,category=a,axis=1)

**how many are unique?**

In [13]:
len(df['asn'].unique())

200

In [14]:
df['asn'].unique()[:10]

array(['46887', '14586', '204428', '45090', '132203', '14061', '4134',
       '28361', '64050', '9498'], dtype=object)

In [15]:
len(df['asn_country'].unique())

58

In [16]:
df['asn_country'].unique()

array(['US', 'BG', 'CN', 'BR', 'SG', 'IN', 'MY', 'HK', 'RU', 'ES', 'CO',
       'TH', 'AU', 'ID', 'FR', 'CA', 'GB', 'PA', 'VN', 'HR', 'KR', 'BO',
       'TW', 'AR', 'VE', 'PH', 'SC', 'IT', 'PE', 'SV', 'TR', 'JP', 'PT',
       'MX', 'ZA', 'NG', 'CL', 'GT', 'DE', 'TN', 'MN', 'BY', 'IR', 'PS',
       'SE', 'PK', 'NL', 'CH', 'LA', 'MM', 'PL', 'KE', 'BE', 'EC', '',
       'IS', 'GR', 'EG'], dtype=object)

In [17]:
countries = df['asn_country'].unique()

In [18]:
df[['asn','asn_desc','Source IP']].value_counts()

asn    asn_desc                                                                    Source IP    
45090  CNNIC-TENCENT-NET-AP Shenzhen Tencent Computer Systems Company Limited, CN  192.168.23.89    173
14061  DIGITALOCEAN-ASN, US                                                        192.168.23.89     91
4134   CHINANET-BACKBONE No.31,Jin-rong Street, CN                                 192.168.23.89     33
16276  OVH, FR                                                                     192.168.23.89     26
56046  CMNET-JIANGSU-AP China Mobile communications corporation, CN                192.168.23.89     15
                                                                                                   ... 
27699  TELEFONICA BRASIL S.A, BR                                                   192.168.23.89      1
27843  OPTICAL TECHNOLOGIES S.A.C., PE                                             192.168.23.89      1
27882  Telefonica Celular de Bolivia S.A., BO                          

#### add AS Features

 Add booleans if the connection is either a known invasive tech company or in the sketchy country list.

In [19]:
df['sketchy'] = df['asn_country'].apply(is_sketchy)
df['iscorp'] = df['asn_desc'].apply(is_corp)
df['sketchy_provider'] = df['asn_desc'].apply(is_sketchy_provider)

### Network sanity

Are DNS/Reverse protocols handled in a friendly way?

- reverse pointers
- DNS entries

####  hostname lookups

Not only is this helpful to get an eyeball sense of where streams are going, it gives you a good idea of what doesn't have a name. 

**This takes awhile**

Garbage connections often don't have a DNS record. The timeout process makes this table take awhile to build

In [ ]:
df['Source Name'] = df['Source IP'].apply(iplookup)
df['Destination Name'] = df['Destination IP'].apply(iplookup)


In [ ]:
df['Source Name'].apply(isip)

In [ ]:
df['src_ptr'] = df['Source Name'].apply(has_ptr)
df['dst_ptr'] = df['Destination Name'].apply(has_ptr)
df['src_dns'] = df['Source Name'].apply(has_dns)
df['dst_dns'] = df['Destination Name'].apply(has_dns)


**did we miss any?**

In [ ]:
# weird entry - whois returns the AS info, but no description or prefix
# AS      | IP               | BGP Prefix          | CC | Registry | Allocated  | AS Name
# NA      | 69.195.171.128   | NA                  | US | arin     | 2017-09-18 | NA
# From Hurricane Electric - Twitter:
# AS13414 IRR Valid 69.195.171.0/24 Twitter Inc.
df[df['asn'] == 'NA']

### Checking for fail2ban entries

https://www.fail2ban.org/wiki/index.php/Main_Page

If you aren't familiar, fail2ban scans log files (e.g. /var/log/apache/error_log) and bans IPs that show malicious signs -- too many password failures, seeking for exploits, etc. If something hammers the logs enough to trigger a fail2ban entry this adds suspicion to the originating connection. 

A return of ICMP 3 (unreachable) means that the host was caught by fail2ban so we can filter on that from the logs.





In [ ]:
# here we show the unique protocols available in our test
len(df['Port:Protocol:Service'].unique())

In [ ]:
# and here's a count of which protocols are represented in our sample
df['Port:Protocol:Service'].value_counts()

In [ ]:
#services = {'icmp':3,'ssh':22,'smtp':25,'dns':53,'ssl':443,'http':80}
services = ['icmp','ssh','smtp','dns','ssl','http']

In [ ]:
# multiple match list
# [s for s in my_list if any(xs in s for xs in matchers)] # greedy - returns too much
# {s for s in my_list for xs in matchers if xs in s}

In [ ]:
# add fail2ban hit feature
df['fail2ban'] = df['Port:Protocol:Service'].apply(f2b_marked)

### some simple aggregated term analysis


In [ ]:
# sketchy is false
df[~df['sketchy']][:3]

In [ ]:
# connections flagged by fail2ban with no DNS entry
df[(~df['dst_dns'])&(df['fail2ban'])]

### Extracts using the flags

Now we can use pandas and the features to test the output.

In [ ]:
# all providers where connection has no dst_ptr or dst_dns and has a fail2ban hit
df[(~df['dst_ptr'])&(~df['dst_dns']) &(df['fail2ban'])].asn_desc.unique()

In [ ]:
# grab the value of the asn_desc where the item not sketchy
df[(~df['sketchy']) & (~df['src_dns'])][:3]

In [ ]:
df[(~df['sketchy']) &(df['fail2ban'])][:3]

### what AS regions get the most traffic?

In [ ]:
df[['asn','asn_desc','Source IP']].value_counts()

In [ ]:
df[['asn','asn_desc','Source IP']][:11].value_counts()

### Stats analysis

In [ ]:
# describe the stats
df.describe()

mean score

In [ ]:
df['Score'].mean().round(3)

mean duration in ms

In [ ]:
df['Duration'].mean().round(3)

relative item correlation

In [ ]:
df.corr()

#### adding a heatmap to the correlation

This data doesn't have corporate returns or ASs from the sketchy provider map. 
TODO: sort out sketchy providers from the data at the start of the definitions

In [ ]:
fig= plt.figure(figsize=(15,8))
sns.heatmap(df.corr(), linewidths=.1, linecolor='black')

### adding viz and stats

What are the most prevalent AS Numbers?

In [ ]:
df['asn'][:30].value_counts().plot(kind='bar')

Where are they coming from?

In [ ]:
df['asn_desc'][:30].value_counts().plot(kind='bar')

What countries account for the most traffic?

In [ ]:
df['asn_country'][:10].value_counts().plot(kind='bar')

Is there a correlation between average bytes and number of connections?

In [ ]:
df[['Avg Bytes','Connections']][:10].plot()

**whats the relative occurrance of high beacon traffic?**

How about the occurrange of high beaconish traffic?

In [ ]:
df[['Score']][:30].plot(y='Score')

In [ ]:
sns.distplot(df['Score'])

**how about long duration**

In [ ]:
sns.distplot(df['Duration'])

#### how about services?


In [ ]:
# The highest hits is fail2ban attempting to quash traffic, so we'll remove the ICMP entries
df[~df['Port:Protocol:Service'].str.contains('icmp')]['Port:Protocol:Service'].value_counts().plot(kind='bar')

### High Beaconish Originators

What are the AS originators with over 80% beacon traffic and what country are they from?

In [ ]:
df[(df['Score'] >.80)][3:]['asn_desc']

In [ ]:
df['asn_country'].value_counts()

In [ ]:
# A slightly deeper dive into Chinese traffic
df[df['asn_country'] =='CN']

### connections where duration value is short and beaconish is high

- only get low duration connections which exhibit above %75 beaconism 

In this case, there a bunch of ICMP messages originating from my host heading to (mostly) China. If fail2ban wasn't running this might be cause for further investigation, but fail2ban sends ICMP type 3 packets to an originator when it gets jailed. We're catching this upstream in the fail2ban column. 

In [ ]:
# What are the relative duration statistics?
df['Duration'].describe()

In [ ]:
# What is the relative score distribution?
df['Score'].describe()

In [ ]:
# looking at raw duration length values
df['Duration'].sort_values()

**Start drilling down**

Find the mean of all the Duration values. Use the Mean to determine how ordinary the duration of the traffic is

In [ ]:
df['Duration'].mean()

In [ ]:
# Show only durations below the mean
df[df['Duration'] < df['Duration'].mean()][:10]

**what are connections where duration is below a particular quantile?**

In [ ]:
df[df['Duration'] < df['Duration'].quantile(.2)][:10]

How about where the duration is less than the .2 quantile, but with scores higher than 75%?

In [ ]:
print(len(df[(df['Duration'] < df['Duration'].quantile(.2)) & (df['Score'] > .75)]))
df[(df['Duration'] < df['Duration'].quantile(.2)) & (df['Score'] > .75)][:3]

**if anything is left originating to an external host that's not fail2ban there is something to dig further into**



In [ ]:
# if anything is left that's not fail2ban there is something to dig further into
df[(df['Duration'] < df['Duration'].quantile(.2)) & (df['Score'] > .75)&(~df['fail2ban']) & (df['Destination IP'] != '192.168.23.89') ][:10]

**is anything not originating from my ip?**

In [ ]:
# is anything not originating from my ip?
df[(df['Duration'] < df['Duration'].quantile(.2)) & (df['Score'] > .75)]['Source IP'].unique()

**look at only non-fail2ban items where Duration is in the upper quantile, Score is greater than .75 and originates from my server**

Nothing in this batch.

In [ ]:
# look at only non-fail2ban items where Duration is in the upper quantile, Score is greater than .75 and originates from my server
# nothing here - so it looks like beaconish activity here is fail2ban related (handled by )
df[(df['Duration'] < df['Duration'].quantile(.2)) & (df['Score'] > .75)& (~df['fail2ban']) &(df['Source IP'].str.contains('192.168.23.89'))]

In [ ]:
df[(df['Duration'] < df['Duration'].quantile(.2)) & (df['Score'] > .75)&(df['Source IP'] != '192.168.23.89')][:10]

**where are ssh connections coming/going?**

everything appears to be incoming, so we aren't launching any attacks

In [ ]:
# where are ssh connections coming/going?
# everything appears to be incoming, so we aren't launching any attacks
df[(df['Port:Protocol:Service'].str.contains('ssh')) & (df['Source IP'] != "10.4.86.55")]['ConnString'].unique()

**show all unique source names with scores above 80%**

In [ ]:
# show all unique source names with scores above 80%
df[df['Score']> .8]['Source Name'].unique()

### Blacklists

Adding a blacklist heuristic. Most of rita-bl seems borked right now (stale data, backends offline, etc). In the meantime, lets get visibility using the spamhaus data.

In [ ]:
# importing a custom, line-delimited list
blacklistraw = open('../records/20210827154850_blacklisted_ips.txt','r').readlines()
blacklist = [x.strip('\n') for x in blacklistraw]

In [ ]:
# add feature
df['blacklisted'] = df['Destination IP'].apply(in_blacklist)

In [ ]:
# are there any hits?
df[df['blacklisted']]

### Heuristics 
#### show the tally points

Here we want to score based on the conditions. Some things are bad if they're True (sketchy TLD like Russia or China) some are bad if they're False (no reverse_ptr). Scoring needs a scale: some things are inherently worse (domain is the source of an attack in the wild) and some are not (reverse DNS).

reasons to believe the traffic is not good (this could use expansion)
sketchy - if True (the connection is from a poorly managed country tld) add 3
fail2ban - if True (domain is spawning attacks in the wild) add 3
sketchy_provider - if True then bad - add 3

formal laziness: 
src_ptr - if they are the source and his is false, then bad 2
dst_ptr - if they are the dst and it is false, then bad 2
src_dns - if they are the source and his is false, then bad 2
dst_dns - if they are the dst and it is false, then bad 2

corporate canaries:
iscorp - corporate canaries (apple, google, microsoft, etc). If true, then bad (though probably harmless). 1  



so what I need is:
- a feature that lets me know if src/dst is important for ptr and dns
- a function that returns the value if the feature is present for each item and then tallies a score to be added as a feature.

In [ ]:
# we're tallying on these columns
df[['sketchy','src_ptr','dst_ptr','iscorp','sketchy_provider','src_dns','dst_dns','fail2ban', 'blacklisted']][:5]

In [ ]:
# using tally_total
df[['bscore','asn','sketchy','src_ptr','dst_ptr','iscorp','sketchy_provider','src_dns','dst_dns','fail2ban','blacklisted']][:10].apply(tally_total,axis=1)

**adding heuristics score (bscore)**

In [ ]:
# full scoring
# using tally_total
df['bscore'] = df[['bscore','asn','sketchy','src_ptr','dst_ptr','iscorp','sketchy_provider','src_dns','dst_dns','fail2ban','blacklisted']].apply(tally_total,axis=1)

#### Now create a total score

Now to make the single value that represents RITA's statistical analysis score ('Score') and our heuristical score ('bscore'). For the moment it seems that Score * bscore is useful because Score is a Percentage which should scale the raw heuristical tally nicely.

In [ ]:
df[['Score','bscore']][:10].apply(total_score,axis=1)

**create the new feature**

In [ ]:
df['total_score'] = df[['Score','bscore']].apply(total_score,axis=1)

##### Sort the list by total score 

Non-corp connections should bubble up and we should only be grading on bad actors and malware.

In [ ]:
df[:10].sort_values(by='total_score', ascending=False)

In [ ]:
sns.distplot(df['Score'])

In [ ]:
sns.distplot(df['bscore'])

In [ ]:
sns.distplot(df['total_score'])

### Summarize reporting

In [ ]:
df[['Score','bscore', 'total_score','Source Name', 'Destination Name', 'Connections', 'Avg Bytes','asn_desc','asn_country']].sort_values(by='total_score', ascending=False)

In [ ]:
# Final correlation
df.corr()

In [ ]:
# final heatmap
fig= plt.figure(figsize=(15,8))
sns.heatmap(df.corr(), linewidths=.1, linecolor='black')

## Final

And that's the basic process. I want to be able to scan through connections at least daily, then export the outputs to a datastore or to reporting for followup.



